In [ ]:
!pip install gym
!apt-get update
!apt-get install swig
!pip install box2d box2d-kengz
!pip install pybullet

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.co

In [ ]:
from tkinter.messagebox import NO
from turtle import forward
from unicodedata import name
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np
from collections import deque
import random
from pathlib import Path

In [ ]:

class ActionNoise:
    def __init__(self, mu, sigma=0.15, theta=0.15, dt=1e-2):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x_prev = np.zeros_like(mu)
        self.reset()

    def __call__(self):
        self.x_prev += self.theta * (self.mu - self.x_prev) + self.sigma * np.sqrt(
            self.dt
        ) * np.random.normal(size=self.mu.shape)
        return self.x_prev

    def reset(self):
        self.x_prev = np.zeros_like(self.mu)


class ReplayBuffer:
    def __init__(self, maxlen=100000, batch_size=32, use_cuda=True) -> None:
        self.buffer = deque(maxlen=maxlen)
        self.batch_size = batch_size
        self.use_cuda = use_cuda
        pass

    def save_experience(self, state, action, reward, next_state, done):
        state = (
            torch.FloatTensor(state)
            if not self.use_cuda
            else torch.FloatTensor(state).cuda()
        )
        action = (
            torch.FloatTensor(action)
            if not self.use_cuda
            else torch.FloatTensor(action).cuda()
        )
        reward = (
            torch.FloatTensor(np.array([reward]))
            if not self.use_cuda
            else torch.FloatTensor(np.array([reward])).cuda()
        )
        next_state = (
            torch.FloatTensor(np.array(next_state))
            if not self.use_cuda
            else torch.FloatTensor(np.array(next_state)).cuda()
        )
        done = (
            torch.FloatTensor(np.array([1 - np.float32(done)]))
            if not self.use_cuda
            else torch.FloatTensor(np.array([1 - np.float32(done)])).cuda()
        )

        self.buffer.append((state, action, reward, next_state, done))

    def get_sample(self):
        batch = random.sample(self.buffer, self.batch_size)
        state, action, reward, next_state, done = map(torch.stack, zip(*batch))
        return (
            state,
            next_state,
            action.squeeze(),
            reward.squeeze(),
            done.squeeze(),
        )


In [ ]:
class Critic(nn.Module):
    def __init__(
        self,
        beta,
        state_dim,
        action_dim,
        name,
        layer1_size=400,
        layer2_size=300,
        save_dir=None,
    ):
        super(Critic, self).__init__()
        self.use_cuda = torch.cuda.is_available()
        self.device = "cuda" if self.use_cuda else "cpu"
        self.checkpoint_dir = save_dir
        self.name = name

        self.layer1 = nn.Linear(state_dim, layer1_size)
        f1 = 1 / np.sqrt(self.layer1.weight.data.size()[0])
        torch.nn.init.uniform_(self.layer1.weight.data, -f1, f1)
        torch.nn.init.uniform_(self.layer1.bias.data, -f1, f1)
        self.bn1 = nn.LayerNorm(layer1_size)

        self.layer2 = nn.Linear(layer1_size, layer2_size)
        f2 = 1 / np.sqrt(self.layer2.weight.data.size()[0])
        torch.nn.init.uniform_(self.layer2.weight.data, -f2, f2)
        torch.nn.init.uniform_(self.layer2.bias.data, -f2, f2)
        self.bn2 = nn.LayerNorm(layer2_size)

        self.action_layer = nn.Linear(action_dim, layer2_size)
        f3 = 0.003
        self.value_func = nn.Linear(layer2_size, 1)
        torch.nn.init.uniform_(self.value_func.weight.data, -f3, f3)
        torch.nn.init.uniform_(self.value_func.bias.data, -f3, f3)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=beta)
        self.to(self.device)
        pass

    def forward(self, state, action):
        state_value = self.layer1(state)
        state_value = self.bn1(state_value)
        state_value = f.relu(state_value)

        state_value = self.layer2(state_value)
        state_value = self.bn2(state_value)
        state_value = f.relu(state_value)

        action_value = f.relu(self.action_layer(action))
        value = self.value_func(f.relu(torch.add(state_value, action_value)))
        return value

    def save_network(self):
        torch.save(self.state_dict(), self.checkpoint_dir / f"{self.name}.ckpt")

    def load_network(self, load_dir):
        self.load_state_dict(torch.load(load_dir / f"{self.name}.ckpt"))

In [ ]:

class Actor(nn.Module):
    def __init__(
        self,
        alpha,
        state_dim,
        action_dim,
        name,
        layer1_size=400,
        layer2_size=300,
        save_dir=None,
    ):
        super(Actor, self).__init__()
        self.use_cuda = torch.cuda.is_available()
        self.device = "cuda" if self.use_cuda else "cpu"
        self.checkpoint_dir = save_dir
        self.name = name

        self.layer1 = nn.Linear(state_dim, layer1_size)
        f1 = 1 / np.sqrt(self.layer1.weight.data.size()[0])
        torch.nn.init.uniform_(self.layer1.weight.data, -f1, f1)
        torch.nn.init.uniform_(self.layer1.bias.data, -f1, f1)
        self.bn1 = nn.LayerNorm(layer1_size)

        self.layer2 = nn.Linear(layer1_size, layer2_size)
        f2 = 1 / np.sqrt(self.layer2.weight.data.size()[0])
        torch.nn.init.uniform_(self.layer2.weight.data, -f2, f2)
        torch.nn.init.uniform_(self.layer2.bias.data, -f2, f2)
        self.bn2 = nn.LayerNorm(layer2_size)

        f3 = 0.003
        self.actions = nn.Linear(layer2_size, action_dim)
        torch.nn.init.uniform_(self.actions.weight.data, -f3, f3)
        torch.nn.init.uniform_(self.actions.bias.data, -f3, f3)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=alpha)
        self.to(self.device)
        pass

    def forward(self, state):
        x = self.layer1(state)
        x = self.bn1(x)
        x = f.relu(x)

        x = self.layer2(x)
        x = self.bn2(x)
        x = f.relu(x)

        action = torch.tanh(self.actions(x))
        return action

    def save_network(self):
        torch.save(self.state_dict(), self.checkpoint_dir / f"{self.name}.ckpt")

    def load_network(self, load_dir):
        self.load_state_dict(torch.load(load_dir / f"{self.name}.ckpt"))


In [ ]:
class Agent:
    def __init__(
        self,
        alpha,
        beta,
        state_dim,
        save_dir,
        tau=0.001,
        gamma=0.95,
        action_dim=4,
        max_len=1000000,
        layer1=400,
        layer2=300,
        batch_size=64,
    ):
        self.gamma = gamma
        self.tau = tau
        self.burnin = 1e4
        self.curr_step = 0
        self.learn_every = 3
        self.save_ever = 1e5

        self.memory = ReplayBuffer(maxlen=max_len, batch_size=batch_size, use_cuda=True)
        self.save_dir = save_dir
        self.actor = Actor(
            alpha, state_dim, action_dim, "actor", save_dir=self.save_dir
        )
        self.critic = Critic(
            beta, state_dim, action_dim, "critic", save_dir=self.save_dir
        )
        self.target_actor = Actor(
            alpha, state_dim, action_dim, "target_actor", save_dir=self.save_dir
        )
        self.target_critic = Critic(
            beta, state_dim, action_dim, "target_critic", save_dir=self.save_dir
        )

        self.noise = ActionNoise(mu=np.zeros(action_dim))

        # self.update_network(tau=1)
        pass

    def act(self, state, noise=True):
        self.actor.eval()
        state = torch.FloatTensor(state).cuda().unsqueeze(0)
        action = self.actor(state)
        action = action.cpu().data.numpy()
        if noise:
            action += self.noise()
        self.actor.train()
        self.curr_step += 1
        return action

    def remember(self, state, action, reward, next_state, done):
        self.memory.save_experience(state, action, reward, next_state, done)
        pass

    def learn(self):
        if self.curr_step < self.burnin:
            return
        if self.curr_step % self.save_ever == 0:
            self.save_network()
        if self.curr_step % self.learn_every != 0:
            return

        state, next_state, action, reward, done = self.memory.get_sample()

        self.target_actor.eval()
        self.target_critic.eval()
        self.critic.eval()

        target_action = self.target_actor(next_state)
        target_valuefunc = self.target_critic(next_state, target_action)
        valuefunc = self.critic(state, action)

        target = reward.squeeze() + (
            target_valuefunc.squeeze() * done.squeeze() * self.gamma
        )

        target = target.unsqueeze(1).float()

        self.critic.train()
        self.critic.optimizer.zero_grad()
        critic_loss = f.mse_loss(valuefunc, target)
        critic_loss.backward()
        self.critic.optimizer.step()
        self.critic.eval()

        self.actor.optimizer.zero_grad()
        ac = self.actor(state)
        self.actor.train()
        ac_loss = -self.critic(state, ac).mean()
        ac_loss.backward()
        self.actor.optimizer.step()

        self.update_network()

    def update_network(self, tau=None):
        if tau is None:
            tau = self.tau
        actor_params = self.actor.named_parameters()
        critic_params = self.critic.named_parameters()
        target_actor_params = self.target_actor.named_parameters()
        target_critic_params = self.target_critic.named_parameters()

        critic_state_dict = dict(critic_params)
        actor_state_dict = dict(actor_params)
        target_critic_dict = dict(target_critic_params)
        target_actor_dict = dict(target_actor_params)

        for name in critic_state_dict:
            critic_state_dict[name] = (
                tau * critic_state_dict[name].clone()
                + (1 - tau) * target_critic_dict[name].clone()
            )

        self.target_critic.load_state_dict(critic_state_dict)

        for name in actor_state_dict:
            actor_state_dict[name] = (
                tau * actor_state_dict[name].clone()
                + (1 - tau) * target_actor_dict[name].clone()
            )
        self.target_actor.load_state_dict(actor_state_dict)

    def save_network(self):
        self.actor.save_network()
        self.critic.save_network()
        self.target_actor.save_network()
        self.target_critic.save_network()

    def load_network(self, load_dir):
        self.actor.load_network(load_dir)
        self.critic.load_network(load_dir)
        self.target_actor.load_network(load_dir)
        self.target_critic.load_network(load_dir)


In [ ]:
import matplotlib.pyplot as plt

def plotLearning(scores, filename, x=None, window=5):   
    N = len(scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(scores[max(0, t-window):(t+1)])
    if x is None:
        x = [i for i in range(N)]
    plt.ylabel('Score')       
    plt.xlabel('Game')                     
    plt.plot(x, running_avg)
    plt.savefig(filename)

In [ ]:
import numpy as np
from pathlib import Path
import gym
from tqdm.auto import tqdm, trange 
import datetime

In [ ]:
save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)

In [ ]:
env = gym.make('BipedalWalker-v3')
state = env.reset()
agent = Agent(alpha = 0.0001,beta= 0.001, tau=0.001,state_dim = env.observation_space.shape[0],action_dim= env.action_space.shape[0],save_dir=save_dir)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
np.random.seed(0)
episodes = 2500
score_history = []
with trange(episodes) as t:
    for episode in t:
        state = env.reset()
        done = False
        score = 0
        while not done:
            action = agent.act(state)
            n_state,reward,done,info = env.step(action[0])
            agent.remember(state=state,action=action,reward=reward,next_state=n_state,done=done)
            agent.learn()
            state = n_state
            score+=reward
        score_history.append(score)
        t.set_postfix(str = f"Steps:{agent.curr_step} Episode: {episode} Score: {score} Score_avg {np.mean(score_history[-100:])}")
    env.close()  

plotLearning(score_history, 'Reward Graph', window = 100)

  0%|          | 0/2500 [00:00<?, ?it/s]